In [ ]:
# import settings and functions
%run ./../../imports.ipynb

In [ ]:
scal = np.genfromtxt('../scaling.in')
scaling = scal
print(scaling)

# scaling factor for the dataset
# max value of the dataset
# scaling_option = "max"
# std value of the dataset
#scaling_option = "std"
# use the existing scaling factor outside this folder
scaling_option = "existing"

In [ ]:
plotting = True
# csv_name = "dataset_regular_semidense"

# sparse dataset
csv_name = "dataset_regular_sparse"
count_T = 50
# # semidense dataset
# csv_name = "dataset_regular_semidense"
# count_T = 100
# # dense dataset
# csv_name = "dataset_regular_dense"
# count_T = 500

noise = 0.0



In [ ]:
a = 134.0
b = -0.1047
c = 3.719 *10**(-5)


In [ ]:
q_star = []
g_star = []
T_star = []

In [ ]:
T_min = 400
T_max = 1100
G_max = 40
G_min = -G_max

eye_distance = 1.3

In [ ]:
q_star_x = []
g_star_x = []
q_star_y = []
g_star_y = []
T_star = []
for T in np.linspace(T_min, T_max, count_T):
    for g_x in np.linspace(G_min, G_max, count_T):
        for g_y in np.linspace(G_min, G_max, count_T):
            T_star.append(T)
            g_star_x.append(g_x)
            g_star_y.append(g_y)
            q_star_x.append(-(b*T+a+c*T*T)*g_x)
            q_star_y.append(-(b*T+a+c*T*T)*g_y)
    


## Dataset and scaling

In [ ]:
# create pandas dataframe for the dataset with names gradx grady fluxx fluxy value with no index
data = pd.DataFrame({'gradx': g_star_x, 'grady': g_star_y, 'fluxx': q_star_x, 'fluxy': q_star_y, 'value': T_star})
# data.to_csv('dataset_regular.csv', index=False)

# size of the dataset
print(data.shape[0])
headers = data.columns.values
print(headers)


In [ ]:
# save dataset
data.to_csv(csv_name+'.csv', index=False)

In [ ]:
max_vals = []
val_scale_scale = 0.1

max_g_star_x = np.array(g_star_x).max()
max_vals.append(max_g_star_x)
max_g_star_y = np.array(g_star_y).max()
max_vals.append(max_g_star_y)
max_q_star_x = np.array(q_star_x).max()
max_vals.append(max_q_star_x)
max_q_star_y = np.array(q_star_y).max()
max_vals.append(max_q_star_y)
max_T_star = np.array(T_star).max()
max_vals.append(max_T_star)

max_grad = 0
max_flux = 0
max_val = max_vals[4]

for i in [0,1]:
    if max_vals[i] > max_grad:
        max_grad = max_vals[i]
    if max_vals[i+2] > max_flux:
        max_flux = max_vals[i+2]

print(max_vals)
print(max_grad)
print(max_flux)
print(max_val)

if scaling_option == "max":
    scaling = [max_grad/max_flux, max_grad/max_val * val_scale_scale]

print(scaling)


In [ ]:
std_vals = []

std_g_star_x = np.array(g_star_x).std()
std_vals.append(std_g_star_x)
std_g_star_y = np.array(g_star_y).std()
std_vals.append(std_g_star_y)
std_q_star_x = np.array(q_star_x).std()
std_vals.append(std_q_star_x)
std_q_star_y = np.array(q_star_y).std()
std_vals.append(std_q_star_y)
std_T_star = np.array(T_star).std()
std_vals.append(std_T_star)

print(max_vals)
print(std_vals)

if scaling_option == "std":
    scaling = [std_vals[0]/std_vals[2], std_vals[0]/std_vals[4] * val_scale_scale]

print(scaling)

In [ ]:
scaled_data = data.copy()
scaled_data["fluxx"] = data["fluxx"]*scaling[0]
scaled_data["fluxy"] = data["fluxy"]*scaling[0]
scaled_data["value"] = data["value"]*scaling[1] 

# print(scaled_data)

new_max_vals = []
for header in headers:
    new_max_vals.append(scaled_data[header].max())

print(new_max_vals)

In [ ]:
pd.DataFrame(scaled_data).to_csv(csv_name+"_scaled.csv", index=False)
f = open("scaling.in", "w")
f.write(str(scaling[0])+"\n")
f.write(str(scaling[1]))
f.close()

# Plotting 

In [ ]:
if plotting == False:
    np.ll

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Create a surface plot
fig = go.Figure()

color1 = 'gray'

# surface_trace = go.Surface(
#     x=T_star_s,
#     y=g_star_s,
#     z=q_star_s,
#     opacity = .2,
#     colorscale=[[0, color1], [1,color1]],
#     showscale=False
#     # colorscale="Bluered",
#     # surfacecolor=T_star_c,
#     # colorbar=dict(thickness=20),
# )

fig.add_trace(go.Scatter3d(x=T_star,
                           y=g_star_x,
                           z=q_star_x,
                           mode='markers',
                           marker={
                               'size': 1,
                               'opacity': 0.8,
                               'color':T_star,
                               'colorscale': custom_colorscale_temperature,
                               'colorbar':dict(thickness=20, title="Temperature")
                           },
                           ))


# fig.add_trace(surface_trace)

fig.update_layout(
    scene=dict(
        xaxis_title='Temperature',
        yaxis_title='Gradient x',
        zaxis_title='Flux x',
        camera=dict(
            eye=dict(x=1.5*eye_distance, y=1.5*eye_distance, z=0.8*eye_distance)
        )
    ),
    font_family='Times New Roman',
    font_size=15,
    width=700,
    margin=dict(r=20, b=10, l=10, t=10),
    scene_aspectmode='cube'
)

fig.show()

fig.write_image(csv_name+'_TGQ.png', scale=2)

import plotly.io as pio
pio.write_image(fig, csv_name+"_TGQ.svg", scale=2)

In [ ]:
# 3D scatter plot for G_x, G_y, and Q_y
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(data["grady"], data["value"], data["fluxy"], c=data["value"], s=1, cmap=custom_cmap_temperature)
cbar = fig.colorbar(sc, pad=0.2)
ax.set_xlabel(r'$g_y^*$', labelpad=10)
ax.set_ylabel(r'$T^*$', labelpad=10)
ax.set_zlabel(r'$q_y^*$', labelpad=10)
cbar.set_label(r'Temperature ' + r'$T^*$')
plt.tight_layout()
# plt.savefig('dataset_3D_from_ana_loop.pdf')
plt.savefig(csv_name+'_TGQ_mpl.png')
plt.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Your existing code to add actual data points
fig.add_trace(go.Scatter3d(x=g_star_y,
                           y=g_star_x,
                           z=q_star_x,
                           mode='markers',
                           marker={
                               'size': 1,
                               'opacity': 0.8,
                               'color':q_star_y,
                               'colorscale': custom_colorscale_flux,
                               'colorbar':dict(thickness=20, title="Flux y")
                           },
                           ))

# Your existing code to update layout and show the figure
fig.update_layout(scene=dict(
    xaxis_title='Gradient y',
    xaxis_title_font_size=20,
    yaxis_title='Gradient x',
    yaxis_title_font_size=20,
    zaxis_title='Flux x',
    zaxis_title_font_size=20,
    camera=dict(eye=dict(x=1.5*eye_distance, y=1.5*eye_distance, z=0.8*eye_distance))
    ),
    font_family='Times New Roman',
    font_size=15,
    width=700,
    margin=dict(r=20, b=10, l=10, t=10),
    scene_aspectmode='cube'
)

fig.show()

fig.write_image(csv_name+'_GGQ.png', scale=2)

import plotly.io as pio
pio.write_image(fig, csv_name+"_GGQ.svg", scale=2)

In [ ]:
# 3D scatter plot for G_x, G_y, and Q_y
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(data["gradx"], data["grady"], data["fluxx"], c=data["fluxy"], s=1, cmap=custom_cmap_flux)
cbar = fig.colorbar(sc, pad=0.2)
ax.set_xlabel(r'$g_x^*$', labelpad=10)
ax.set_ylabel(r'$g_y^*$', labelpad=10)
ax.set_zlabel(r'$q_x^*$', labelpad=10)
ax.set_xlim(-G_max, G_max)
ax.set_ylim(-G_max, G_max)
ax.set_zlim(-4000,4000)
cbar.set_label(r'Flux ' + r'$q_y^*$')
cbar.set_ticks([-3000, -2000, -1000, 0, 1000, 2000, 3000])
plt.tight_layout()
# plt.savefig('dataset_3D_from_ana_loop.pdf')
plt.savefig(csv_name+'_GGQ_mpl.png')
plt.show()

# x=g_star_y,
#                            y=g_star_x,
#                            z=q_star_x,

In [ ]:
k = [-q/g for q,g in zip(data["fluxy"], data["grady"]) if abs(g) > 0.000001 ]

# g = [g for g in data["grady"] if abs(g) > 0.000001]
T = [T for T,g in zip(data["value"], data["grady"]) if abs(g) > 0.000001]
# q = [q for q,g in zip(data["fluxy"], data["grady"]) if abs(g) > 0.000001]

In [ ]:
# k = []
# for q_in, g_in in zip(data["fluxy"], data["grady"]):
#     k.append(-q_in/g_in)
    
# k = [-q/g for q,g in zip(data["fluxy"], data["grady"]) ]
fig, ax = plt.subplots()
plt.grid(True, ls=':')
plt.scatter(T, k, c=T, s=1, cmap=custom_cmap_temperature)

cbar = plt.colorbar()
# if labels:
plt.xlabel('Temperature ' + r'$T$')
plt.ylabel('Conductivity ' + r'$k$')
cbar.set_label('Temperature ' + r'$T$')
    
plt.tight_layout()
plt.savefig(csv_name+'_conductivity.png', dpi=300,facecolor="none", edgecolor='none')
plt.savefig(csv_name+'_conductivity.svg')
plt.show()

In [ ]:
# T = np.linspace(T_min, T_max, 500)
# k = (b*T+a+c*T*T)

# labels = True

# plt.scatter(T, k, c=T, cmap=custom_cmap_temperature)

# cbar = plt.colorbar()
# if labels:
#     plt.xlabel('Temperature ' + r'$T$')
#     plt.ylabel('Conductivity ' + r'$k$')
#     cbar.set_label('Temperature ' + r'$T$')
    
# plt.tight_layout()
# plt.savefig('ana_dataset_conductivity.png', dpi=300,facecolor="none", edgecolor='none')
# plt.savefig('ana_dataset_conductivity.svg')
# plt.show()

In [ ]:
import plotly as Plotly
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=T_star,
                           y=g_star_x,
                           z=q_star_x,
                           mode='markers',
                           marker={
                               'size': 1,
                               'opacity': 0.8,
                               'color':T_star,
                               'colorscale': custom_colorscale_temperature,
                               'colorbar':dict(thickness=20)
                           },
                           ))

fig.update_layout(scene=dict(
    xaxis_title='Temperature',
    # xaxis_title='Gradient y',
    # xaxis_title_font_color='red',
    xaxis_title_font_size=20,
    yaxis_title='Gradient x',
    # yaxis_title_font_color='green',
    yaxis_title_font_size=20,
    zaxis_title='Flux x',
    # zaxis_title_font_color='blue',
    zaxis_title_font_size=20,

    ),
    font_family='Times New Roman',
    font_size=15,
    width=700,
    margin=dict(r=20, b=10, l=10, t=10),
    scene_aspectmode='cube'
    )

fig.show()

# import plotly.io as pio
# #save a figure of 300dpi, with 1.5 inches, and  height 0.75inches
# pio.write_image(fig, "test.svg", width=1.5*300, height=0.75*300, scale=1)



In [ ]:
q_star_s = []
g_star_s = np.linspace(G_min, G_max, count_T + 1)
T_star_s = np.linspace(T_min, T_max, count_T + 1)
for T in T_star_s:
    for g_x in g_star_s:
        q_star_s.append(-(b*T+a+c*T*T)*g_x)

q_star_s = np.array(q_star_s).reshape((count_T + 1, count_T + 1))

In [ ]:
q_star_s = np.zeros((count_T + 1, count_T + 1))
g_star_s = np.linspace(G_min, G_max, count_T + 1)
T_star_s = np.linspace(T_min, T_max, count_T + 1)
T_star_c = np.zeros((count_T + 1, count_T + 1))

for i in range(count_T+1):
    for j in range(count_T+1):
        q_star_s[j][i]=(-(b*T_star_s[i]+a+c*T_star_s[i]*T_star_s[i])*g_star_s[j])
        T_star_c[j][i] = T_star_s[i]

print(q_star_s)
print(T_star_c)

In [ ]:
# import numpy as np
# import plotly.graph_objects as go

# # Create a surface plot
# fig = go.Figure()

# color1 = 'gray'

# surface_trace = go.Surface(
#     x=T_star_s,
#     y=g_star_s,
#     z=q_star_s,
#     opacity = .2,
#     colorscale=[[0, color1], [1,color1]],
#     showscale=False
#     # colorscale="Bluered",
#     # surfacecolor=T_star_c,
#     # colorbar=dict(thickness=20),
# )

# fig.add_trace(go.Scatter3d(x=T_star,
#                            y=g_star_x,
#                            z=q_star_x,
#                            mode='markers',
#                            marker={
#                                'size': 1,
#                                'opacity': 0.8,
#                                'color':T_star,
#                                'colorscale': custom_colorscale_temperature,
#                                'colorbar':dict(thickness=20, title="Temperature")
#                            },
#                            ))


# fig.add_trace(surface_trace)

# fig.update_layout(
#     scene=dict(
#         xaxis_title='Temperature',
#         yaxis_title='Gradient x',
#         zaxis_title='Flux x',
#         camera=dict(
#             eye=dict(x=1.5*eye_distance, y=1.5*eye_distance, z=0.8*eye_distance)
#         )
#     ),
#     font_family='Times New Roman',
#     font_size=15,
#     width=700,
#     margin=dict(r=20, b=10, l=10, t=10),
#     scene_aspectmode='cube'
# )

# fig.show()

# fig.write_image(csv_name+'TGQ.png', scale=2)

# import plotly.io as pio
# pio.write_image(fig, csv_name+"TGQ.svg", scale=2)

In [ ]:
# import numpy as np
# import plotly.graph_objects as go

# # Create a surface plot
# fig = go.Figure()

# color1 = 'gray'

# surface_trace = go.Surface(
#     x=T_star_s,
#     y=g_star_s,
#     z=q_star_s,
#     opacity = .2,
#     colorscale=[[0, color1], [1,color1]],
#     showscale=False
#     # colorscale="Bluered",
#     # surfacecolor=T_star_c,
#     # colorbar=dict(thickness=20),
# )

# fig.add_trace(go.Scatter3d(x=T_star,
#                            y=g_star_x,
#                            z=q_star_x,
#                            mode='markers',
#                            marker={
#                                'size': 1.5,
#                                'opacity': 0.8,
#                                'color':T_star,
#                                'colorscale': custom_colorscale_temperature,
#                                'cmin': 400,  # Set minimum colormap limit here
#                                'cmax': 1100,  # Set maximum colormap limit here
#                             #    'colorbar':dict(thickness=20, title="Temperature")
#                            },
#                            ))


# fig.add_trace(surface_trace)

# fig.update_layout(
#     scene=dict(
#         xaxis_title='Temperature',
#         yaxis_title='Gradient x',
#         zaxis_title='Flux x',
#         xaxis=dict(range=[800, 830]),  # Set x-axis range limits here
#         yaxis=dict(range=[20, 30]),  # Set y-axis range limits here
#         zaxis=dict(range=[-2200, -1400]),  # Set z-axis range limits here
#         camera=dict(
#             eye=dict(x=1.5*eye_distance, y=1.5*eye_distance, z=0.8*eye_distance)
#         )
#     ),
#     font_family='Times New Roman',
#     font_size=15,
#     width=700,
#     margin=dict(r=20, b=10, l=10, t=10),
#     scene_aspectmode='cube'
# )

# fig.show()

# fig.write_image(csv_name+'surface_TGQ_noisy.png', scale=2)

# import plotly.io as pio
# pio.write_image(fig, csv_name+"surface_TGQ_noisy.svg", scale=2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Create a surface plot
fig = go.Figure()

color1 = 'gray'

fig.add_trace(go.Scatter3d(x=T_star,
                           y=g_star_x,
                           z=g_star_y,
                           mode='markers',
                           marker={
                               'size': 1,
                               'opacity': 0.8,
                               'color':q_star_x,
                               'colorscale': custom_colorscale_flux,
                               'colorbar':dict(thickness=20, title="Flux x")
                           },
                           ))

fig.update_layout(
    scene=dict(
        xaxis_title='Temperature',
        yaxis_title='Gradient x',
        zaxis_title='Gradient y',
        camera=dict(
            eye=dict(x=1.5*eye_distance, y=1.5*eye_distance, z=0.8*eye_distance)
        )
    ),
    font_family='Times New Roman',
    font_size=15,
    width=700,
    margin=dict(r=20, b=10, l=10, t=10),
    scene_aspectmode='cube'
)

fig.show()

fig.write_image(csv_name+'_TGG.png', scale=2)
pio.write_image(fig, csv_name+"_TGG.svg", scale=2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Create a surface plot
fig = go.Figure()

color1 = 'gray'

fig.add_trace(go.Scatter3d(x=T_star,
                           y=q_star_x,
                           z=q_star_y,
                           mode='markers',
                           marker={
                               'size': 1,
                               'opacity': 0.8,
                               'color':g_star_x,
                               'colorscale': custom_colorscale_grad,
                               'colorbar':dict(thickness=20, title="Gradient x")
                           },
                           ))

fig.update_layout(
    scene=dict(
        xaxis_title='Temperature',
        yaxis_title='Flux x',
        zaxis_title='Flux y',
        camera=dict(
            eye=dict(x=1.5*eye_distance, y=1.5*eye_distance, z=0.8*eye_distance)
        )
    ),
    font_family='Times New Roman',
    font_size=15,
    width=700,
    margin=dict(r=20, b=10, l=10, t=10),
    scene_aspectmode='cube'
)

fig.show()

fig.write_image(csv_name+'_TQQ.png', scale=2)
pio.write_image(fig, csv_name+"_TQQ.svg", scale=2)

In [ ]:
# max_vals = []
# val_scale_scale = 0.1

# max_g_star_x = np.array(g_star_x).max()
# max_vals.append(max_g_star_x)
# max_g_star_y = np.array(g_star_y).max()
# max_vals.append(max_g_star_y)
# max_q_star_x = np.array(q_star_x).max()
# max_vals.append(max_q_star_x)
# max_q_star_y = np.array(q_star_y).max()
# max_vals.append(max_q_star_y)
# max_T_star = np.array(T_star).max()
# max_vals.append(max_T_star)

# max_grad = 0
# max_flux = 0
# max_val = max_vals[4]

# for i in [0,1]:
#     if max_vals[i] > max_grad:
#         max_grad = max_vals[i]
#     if max_vals[i+2] > max_flux:
#         max_flux = max_vals[i+2]

# print(max_vals)
# print(max_grad)
# print(max_flux)
# print(max_val)

# scaling = [max_grad/max_flux, max_grad/max_val * val_scale_scale]

# print(scaling)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# # Your data generation code
# count_T = 10  # Assuming you have defined count_T
# a, b, c = 1, 2, 3  # Replace with your actual coefficients

# Create coordinate arrays
g_star_s = np.linspace(G_min, G_max, count_T + 1)
T_star_s = np.linspace(T_min, T_max, count_T + 1)

# Create 2D coordinate arrays using meshgrid
G_star, T_star = np.meshgrid(g_star_s, T_star_s)

# Calculate q_star_s directly using vectorized operations
q_star_s = -(b * T_star + a + c * T_star**2) * G_star

# Create a filled contour plot
# plt.figure(figsize=(10, 8))
plt.figure()
mesh = plt.pcolormesh(G_star, q_star_s, T_star, cmap=custom_cmap_temperature)
plt.colorbar(mesh, label='Temperature')
plt.xlabel('Gradient y')
plt.ylabel('Flux y')
# plt.title('Temperature as a function of Gradient y and Flux y')
plt.tight_layout()
plt.savefig(csv_name+'_surface_temp.png', dpi=300, facecolor="none", edgecolor='none')
plt.savefig(csv_name+'_surface_temp.svg')
plt.savefig(csv_name+'_surface_temp.pdf')
plt.show()


In [ ]:
# import plotly as Plotly
# import plotly.graph_objects as go

# range = max_flux * scaling[0]

# fig = go.Figure()
# fig.add_trace(go.Scatter3d(x=scaled_data["value"],
#                            y=scaled_data["gradx"],
#                            z=scaled_data["fluxx"],
#                            name='material dataset',
#                            mode='markers',
#                            marker={
#                                'size': 2,
#                                'opacity': 0.8,
#                                'color': scaled_data["value"],
#                                'colorbar_title': "Temperature",
#                                'colorscale': "bluered"
#                            },
# ))
# # fig.add_trace(go.Scatter3d(x=far_data["value"],
# #                            y=far_data["gradx"],
# #                            z=far_data["fluxx"],
# #                            name='far away field values',
# #                            mode='markers',
# #                            marker={
# #                                'size': 1,
# #                                'opacity': 0.8,
# # },
# # ))
# # fig.add_trace(go.Scatter3d(x=not_far_data["value"],
# #                            y=not_far_data["gradx"],
# #                            z=not_far_data["fluxx"],
# #                            name='close field values',
# #                            mode='markers',
# #                            marker={
# #                                'size': 3,
# #                                'opacity': 0.8,
# # },
# # ))



# fig.update_layout(scene=dict(
#     xaxis_title='Temperature',
#     yaxis_title='Gradient',
#     zaxis_title='Flux',
#     # xaxis=dict(range=[-range, range]),
#     # yaxis=dict(range=[-range, range]),
#     # zaxis=dict(range=[-range, range])
#     ),
# #     font_family='Times New Roman',
# #     width=700,
# #     margin=dict(r=20, b=10, l=10, t=10),
# #     scene_aspectmode='cube'
#                  )

# fig.show()


